# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [2]:
import pandas as pd
import numpy as np
DATA_FOLDER = './data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [54]:
# To list file and folder dynamically
from os import listdir

# Assign Static Values
EBOLA_DATA_DIR = DATA_FOLDER + "/ebola"

# Assign Name of Countries based on Available data in ebola directory
countries = [c[:-5] for c in listdir(EBOLA_DATA_DIR) if c != "LICENSE"]
ebola_data = []

for country in countries:
    country_data_files = listdir(EBOLA_DATA_DIR + "/" + country + "_data")
    country_data_date_label = list(map(lambda s: s[:-4], countries_data))
    country_data = []
    for country_data_file in country_data_files:
        data_per_country_date = pd.read_csv(EBOLA_DATA_DIR + "/" + country + "_data/" + country_data_file, index_col=[1])
        country_data.append(data_per_country_date.fillna(0))
    country_data = pd.concat(country_data,keys=country_data_date_label)
    ebola_data.append(country_data)

ebola_data = pd.concat(ebola_data,keys=countries)
ebola_data.columns
#guinea = pd.read_csv(EBOLA_DATA_DIR+"/guinea_data/2014-09-04.csv",index_col=[0,1])
#guinea_new = pd.read_csv(EBOLA_DATA_DIR+"/guinea_data/2014-09-07.csv",index_col=[0,1])
#pd.concat([guinea,guinea_new],keys=['09-04','09-07'])




Index(['34 Military Hospital', 'Beyla', 'Bo', 'Bo EMC', 'Boffa', 'Bombali',
       'Bomi County', 'Bong County', 'Bonthe', 'Conakry', 'Coyah', 'Dabola',
       'Dalaba', 'Date', 'Dinguiraye', 'Dubreka', 'Forecariah',
       'Gbarpolu County', 'Grand Bassa', 'Grand Cape Mount', 'Grand Gedeh',
       'Grand Kru', 'Gueckedou', 'Hastings-F/Town', 'Kailahun', 'Kambia',
       'Kenema', 'Kenema (IFRC)', 'Kenema (KGH)', 'Kerouane', 'Kindia',
       'Kissidougou', 'Koinadugu', 'Kono', 'Kouroussa', 'Lofa County', 'Lola',
       'Macenta', 'Margibi County', 'Maryland County', 'Montserrado County',
       'Moyamba', 'Mzerekore', 'National', 'Nimba County', 'Nzerekore', 'Pita',
       'Police training School', 'Police traning School', 'Port Loko',
       'Pujehun', 'River Gee County', 'RiverCess County', 'Siguiri',
       'Sinoe County', 'Telimele', 'Tonkolili', 'Totals', 'Unnamed: 18',
       'Western area', 'Western area combined', 'Western area rural',
       'Western area urban', 'Yomou', 'dat

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here